### Laboratorio 5

grupo:
- Gabriela Contreras: 20213
- Paola de León: 20361
- Diego Córdova: 20212

In [6]:
import matplotlib.pyplot as plt
import gymnasium as gym
import numpy as np

In [5]:
import gymnasium as gym

env = gym.make("CliffWalking-v0")

alpha = 0.1
gamma = 0.99
epsilon = 0.1
episodes = 500

max_steps_per_episode = 100

n_actions = env.action_space.n
n_states = env.observation_space.n

# Función epsilon-greedy
def epsilon_greedy_policy(state, Q, epsilon):
    if np.random.rand() < epsilon:
        return np.random.choice(n_actions) # Acción aleatoria
    else:
        return np.argmax(Q[state]) # Acción óptima

### Implementacion de SARSA

### Implementacion de Q-Learning

### Ejecucion de Algoritmos

### Graficos

### Resultados